In [ ]:
%%sh
pygmentize train_function.R

In [ ]:
%%sh
pygmentize serve_function.R

In [ ]:
%%sh
pygmentize main.R

In [ ]:
%%sh
cat Dockerfile

In [ ]:
%%sh
export REGION=us-east-1
aws ecr create-repository --repository-name r-custom --region $REGION

In [ ]:
%%sh
export REGION=us-east-1
export ACCOUNT_ID=`aws sts get-caller-identity --query Account --output text`
docker build -t r-custom:predictor -f Dockerfile .
export IMAGE_ID=`docker images -q r-custom:predictor`
docker tag $IMAGE_ID $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/r-custom:predictor
aws ecr get-login-password --region $REGION | docker login --username AWS --password-stdin $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/r-custom:predictor
docker push $ACCOUNT_ID.dkr.ecr.$REGION.amazonaws.com/r-custom:predictor

In [ ]:
import sagemaker
from sagemaker.estimator import Estimator

print(sagemaker.__version__)

session = sagemaker.Session()
account_id = session.boto_session.client('sts').get_caller_identity()['Account']
region = session.boto_session.region_name

training = 'file://.'
output = 'file://.'

role = sagemaker.get_execution_role()
r_estimator = Estimator(
             image_uri=account_id+'.dkr.ecr.'+region+'.amazonaws.com/r-custom:predictor',
             role=role,
             instance_count=1, 
             instance_type='local',
             output_path=output,
             hyperparameters={
                  'normalize': False
             }
)

r_estimator.fit({'training':training})

In [ ]:
%%sh
tar tvfz model.tar.gz

In [ ]:
r_predictor = r_estimator.deploy(initial_instance_count=1, instance_type='local')

In [ ]:
import pandas as pd

data = pd.read_csv('housing.csv')
data.drop(['medv'], axis=1, inplace=True)
data = data.to_csv(index=False)

r_predictor.serializer = sagemaker.serializers.CSVSerializer()

response = r_predictor.predict(data)

In [ ]:
print(response)

In [ ]:
r_predictor.delete_endpoint()